# **FLISol BA 2021** - Carlos Bustillo

### 0. Consideraciones

Requiere que los archivos 'sim.py', 'simConst.py', 'remoteapi.dll' estén alojados en la misma carpeta que este cuaderno de Jupyter.
<br> Incluir en el script de Coppelia la línea: simRemoteApi.start(19999)

In [2]:
# Se importa las librerías necesarias
import sim


### 1. Establecer la conexión
Utilizaremos las funciones del API Remoto de VREP.
<br> Para más detalles refiérase a la documentación de la librería:
<br> http://www.coppeliarobotics.com/helpFiles/en/remoteApiFunctionsPython.htm

In [3]:
def connect(port):
# Establece la conexión a VREP
# port debe coincidir con el puerto de conexión en VREP
# retorna el número de cliente o -1 si no puede establecer conexión
    sim.simxFinish(-1) # Por si acaso, cierre todas las conexiones abiertas.
    clientID=sim.simxStart('127.0.0.1',port,True,True,2000,5) # Conectarse
    if clientID == 0: print("Conectado a", port)
    else: print("No se pudo conectar")
    return clientID


**Parámetros de simxStart**

number clientID=***simxStart***(string connectionAddress,number connectionPort,boolean waitUntilConnected,boolean doNotReconnectOnceDisconnected,number timeOutInMs,number commThreadCycleInMs)

In [4]:
# Conectarse al servidor de VREP
# *** IMPORTANTE: Ejecutar cada vez que se reinicia la simulación ***
clientID = connect(19999)

Conectado a 19999


### 2. Obtener los manejadores (handlers)</h3>
Un **manejador (handler)** es un número identificador que asigna VREP para cada uno de los elementos de la escena. A través de su manejador se puede hacer referencia a un elemento en específico de la escena.

**Parámetros de simxGetObjectHandle**

 number returnCode,number handle=***simxGetObjectHandle***(number clientID,string objectName,number operationMode)
 
***simx_opmode_blocking*** 
(Modo de bloqueo): Se envía el comando, y la función esperará la respuesta real y la devolverá (si la función no sale de tiempo). La respuesta del comando recibida se eliminará del búfer de la bandeja de entrada (otros modos de operación dejarán sus respuestas de comando en el búfer de la bandeja de entrada

In [5]:
# Obtenemos los manejadores para cada una de las articulaciones del Robot
ret,end_effector=sim.simxGetObjectHandle(clientID,'Dummy',sim.simx_opmode_blocking)

ret,joint1=sim.simxGetObjectHandle(clientID,'MTB_joint1',sim.simx_opmode_blocking)
ret,joint2=sim.simxGetObjectHandle(clientID,'MTB_joint2',sim.simx_opmode_blocking)
ret,joint3=sim.simxGetObjectHandle(clientID,'MTB_joint3',sim.simx_opmode_blocking)
print(joint1, joint2, joint3)

21 22 25


**Prueba para controlar los manejadores**

In [24]:
#import sim
#clientID = connect(19999)

# Asignamos un valor random de prueba
valor = 30

#Definir
ret,joint2=sim.simxGetObjectHandle(clientID,'MTB_joint2',sim.simx_opmode_blocking)

retCode = sim.simxSetJointTargetPosition(clientID, joint2, valor, sim.simx_opmode_oneshot)

<h3> 3. Obtener alguna información adicional</h3>
Utilizando los manejadores, podemos obtener información de los elementos.

In [9]:
# Posición Actual del actuador final
retCode,pos=sim.simxGetObjectPosition(clientID, end_effector, -1, sim.simx_opmode_blocking)
print(pos)

[0.5034672617912292, 0.14492854475975037, 0.5720040798187256]


In [10]:
# Orientación Actual del actuador final
retCode,orient=sim.simxGetObjectOrientation(clientID, end_effector, -1, sim.simx_opmode_blocking)
print(orient)

[-3.1415915489196777, 1.880273316601233e-06, 0.26547425985336304]


In [11]:
# Posición de las articulaciones
returnCode, pos1 = sim.simxGetJointPosition(clientID, joint1, sim.simx_opmode_blocking)
print(pos1)

1.4901158351676713e-07


### 4) A mover el Robot por socket
Utilizando los manejadores, podemos enviar parámetros a los elementos.

In [ ]:
#!/usr/bin/env python3
import socket
import time

# conectamos
clientID = connect(19999)

# Definimos los datos iniciales
HOST0 = '192.168.1.130'       # The server's hostname or IP address
PORT0 = 65430                # The port used by the server

while(True):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s0:
        s0.connect((HOST0, PORT0))
        
        #Dato recibido
        data0 = float(s0.recv(1024))
        #print(data0)
        
        #La simulacion acepta radianes
        dataR0 = data0*0.0175
    
        sim.simxSetJointTargetPosition(clientID, joint1, dataR0, sim.simx_opmode_oneshot)
        #time.sleep(0.2)

<h3> Gráfica de la trayectoria recorrida por la articulación</h3>

<br>Se guardan todas las posiciones (x,y,z) de la trayectoria del gripper en una lista, una vez finalizada la simulación se realiza el gráfico. 
<br>

In [ ]:
import socket
import time

# Conectamos
clientID = connect(19999)

# Posiciones
posiciones = list()

# Definimos los datos iniciales
HOST2 = '192.168.1.132'       # The server's hostname or IP address
PORT2 = 65432                # The port used by the server

while(True):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s2:
        s2.connect((HOST2, PORT2))
        
        #Guardar posicion actual
        retCode,pos=sim.simxGetObjectPosition(clientID, end_effector, -1, sim.simx_opmode_blocking)
        posiciones.append(pos)
        
        #Dato recibido
        data2 = float(s2.recv(1024))
        #print(data2)
        
        #La simulacion acepta radianes
        dataR2 = data2*0.0175
    
        sim.simxSetJointTargetPosition(clientID, end_effector, dataR2, sim.simx_opmode_oneshot)
        #time.sleep(0.2)

In [ ]:
#Graficar posiciones
%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D

#Mostrar posiciones
#print(posiciones)

#Graficar trayectoria recorrida en 3D
fig = plt.figure()
ax = Axes3D(fig)
ax.plot(posiciones[0], posiciones[1], posiciones[2], color="red")
plt.show()

### 5) Conexión microcontrolador (Ej.: Arduino) y Python por serial

Un script en Python envia datos al robot físico
<br>

In [ ]:
#!/usr/bin/env python3
import serial
import socket
import time

# Definimos los datos iniciales
HOST0 = '192.168.1.130'       # The server's hostname or IP address
PORT0 = 65430                # The port used by the server

#Puerto Serial Arduino
puerto = "/dev/ttyUSB1"
arduino = serial.Serial(puerto, 9600)   # create serial object named arduino

while(True):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s0:
        s0.connect((HOST0, PORT0))
        
        #Dato recibido
        data0 = float(s0.recv(1024))
        #print(data0)
        
        arduino.write(bytes(data0,'utf-8'))           # write position to serial port
        reachedPos = str(arduino.readline())            # read serial port for arduino echo
        print(reachedPos)